## Import stuff and load calculations

In [1]:
import matplotlib.patheffects as path_effects
import matplotlib.pyplot as plt
import numpy as np
import os, sys
sys.path.append(f"{os.path.dirname(os.getcwd())}/scripts")

In [2]:
# from creative_and_factual_metrics import *

In [3]:
from sentence_transformers import SentenceTransformer
from diversity_metrics.metrics.model_free_metrics import *
from diversity_metrics.embeddings.models import *
from diversity_metrics.metrics.generalized_diversity import *

In [4]:
llama2_results_5_words = np.load(f"{os.path.dirname(os.getcwd())}/results/creative_factual/llama2_results_5_words.npy", allow_pickle=True).item()
llama2_chat_results_5_words = np.load(f"{os.path.dirname(os.getcwd())}/results/creative_factual/llama2-chat_results_5_words.npy", allow_pickle=True).item()
# pythia_69b_results_5_words  = np.load(f"{os.path.dirname(os.getcwd())}/results/creative_factual/pythia-6.9b_results_5_words.npy", allow_pickle=True).item()
# pythia_69b_sft_results_5_words  = np.load(f"{os.path.dirname(os.getcwd())}/results/creative_factual/pythia-6.9b-sft_results_5_words.npy", allow_pickle=True).item()
# pythia_69b_dpo_results_5_words  = np.load(f"{os.path.dirname(os.getcwd())}/results/creative_factual/pythia-6.9b-dpo_results_5_words.npy", allow_pickle=True).item()
# pythia_69b_ppo_results_5_words  = np.load(f"{os.path.dirname(os.getcwd())}/results/creative_factual/pythia-6.9b-ppo_results_5_words.npy", allow_pickle=True).item()

llama2_results_20_words = np.load(f"{os.path.dirname(os.getcwd())}/results/creative_factual/llama2_results_20_words.npy", allow_pickle=True).item()
llama2_chat_results_20_words = np.load(f"{os.path.dirname(os.getcwd())}/results/creative_factual/llama2-chat_results_20_words.npy", allow_pickle=True).item()
# pythia_69b_results_20_words  = np.load(f"{os.path.dirname(os.getcwd())}/results/creative_factual/pythia-6.9b_results_20_words.npy", allow_pickle=True).item()
# pythia_69b_sft_results_20_words  = np.load(f"{os.path.dirname(os.getcwd())}/results/creative_factual/pythia-6.9b-sft_results_20_words.npy", allow_pickle=True).item()
# pythia_69b_dpo_results_20_words  = np.load(f"{os.path.dirname(os.getcwd())}/results/creative_factual/pythia-6.9b-dpo_results_20_words.npy", allow_pickle=True).item()
# pythia_69b_ppo_results_20_words  = np.load(f"{os.path.dirname(os.getcwd())}/results/creative_factual/pythia-6.9b-ppo_results_20_words.npy", allow_pickle=True).item()

## Functions

In [ ]:
temperatures = [k / 10. for k in range(1, 16)]
temperatures

In [ ]:
from functools import partial

def limit_num_words(sentence, max_num_words):
    return " ".join(sentence.split()[:max_num_words])


def average_pairwise_jaccard(sentences, n=2):
    return np.mean([pairwise_ngram(n, x, y) for x, y in combinations(sentences, 2)])

def self_bleu_smooth(sentences):
    '''
    Calculates the Self-BLEU score for a collection of generated examples (https://arxiv.org/abs/1802.01886)
    :param sentences: List of generated examples
    :return:
    '''

    scores = []
    for i, hypothesis in enumerate(sentences):
        hypothesis_split = hypothesis.strip().split()

        references = [sentences[j].strip().split() for j in range(len(sentences)) if i != j]

        scores.append(sentence_bleu(references, hypothesis_split, smoothing_function=SmoothingFunction().method1))

    return sum(scores) / len(scores)

diversity_metrics = {"selfBleuSmoothed": self_bleu_smooth,
                    "average_pairwise_ncd": lambda sentences: np.mean(get_pairwise_ncd(sentences)),
                     "average_pairwise_jaccard_2": partial(average_pairwise_jaccard, n=2),
                    "average_pairwise_jaccard_3": partial(average_pairwise_jaccard, n=3),
                    "average_pairwise_jaccard_4": partial(average_pairwise_jaccard, n=4),
                    "avg_compression_ratio_full": avg_compression_ratio_full,
                    "avg_compression_ratio_target": avg_compression_ratio_target,
                    "cosine_similarity": None # will be filled in later
                    }

## Plot output diversity metrics

In [ ]:
metric_label = ["Self Bleu Smoothed", "Self Bleu Smoothed", "Average Pairwise NCD", "Average Pairwise NCD", 
               "Average Pairwise Jaccard 2", "Average Pairwise Jaccard 2", "Average Pairwise Jaccard 3", 
               "Average Pairwise Jaccard 3", "Average Pairwise Jaccard 4", "Average Pairwise Jaccard 4", 
               "Average Compression Ratio Full", "Average Compression Ratio Full","Average Compression Ratio Target", 
               "Average Compression Ratio Target", "Cosine Similarity", "Cosine Similarity"]

In [ ]:
creative_factual_full_f = f"{os.path.dirname(os.getcwd())}/results/creative_factual_llama_full.png"

num_metrics = len(diversity_metrics.keys())
fig, axes = plt.subplots(num_metrics, 2, figsize=(20, 5*num_metrics))
# make font bigger
plt.rcParams.update({'font.size': 16})

text_effect = [path_effects.withStroke(linewidth=3, foreground='white')]

i=0
for index, metric in enumerate(diversity_metrics.keys()):

    for col_idx, result_chat in enumerate([llama2_chat_results_20_words, llama2_chat_results_5_words]):
        ax = axes[index, col_idx]

        scores_factual_chat = result_chat["factual"][metric]
        scores_creative_chat = result_chat["creative"][metric]

        result_plm = [llama2_results_20_words, llama2_results_5_words][col_idx]
        scores_factual_plm = result_plm["factual"][metric]
        scores_creative_plm = result_plm["creative"][metric]

        # Plotting for factual scores
        ax.plot(temperatures, scores_factual_chat[:, :, 0].mean(axis=1), label='llama2-chat', marker='o', color="blue")
        ax.fill_between(temperatures, scores_factual_chat[:, :, 0].mean(axis=1) - 2 * scores_factual_chat[:, :, 0].std(axis=1) / np.sqrt(scores_factual_chat.shape[1]), 
                        scores_factual_chat[:, :, 0].mean(axis=1) + 2 * scores_factual_chat[:, :, 0].std(axis=1) / np.sqrt(scores_factual_chat.shape[1]), alpha=0.2, color="blue")

        ax.plot(temperatures, scores_factual_plm[:, :, 0].mean(axis=1), label='llama2', marker='o', color="red")
        ax.fill_between(temperatures, scores_factual_plm[:, :, 0].mean(axis=1) - 2 * scores_factual_plm[:, :, 0].std(axis=1) / np.sqrt(scores_factual_plm.shape[1]), 
                        scores_factual_plm[:, :, 0].mean(axis=1) + 2 * scores_factual_plm[:, :, 0].std(axis=1) / np.sqrt(scores_factual_plm.shape[1]), alpha=0.2, color="red")

        # Plotting for creative scores
        ax.plot(temperatures, scores_creative_chat[:, :, 0].mean(axis=1), label='llama2-chat creative', marker='o', linestyle='--', color="lightblue")
        ax.fill_between(temperatures, scores_creative_chat[:, :, 0].mean(axis=1) - 2 * scores_creative_chat[:, :, 0].std(axis=1) / np.sqrt(scores_creative_chat.shape[1]), 
                        scores_creative_chat[:, :, 0].mean(axis=1) + 2 * scores_creative_chat[:, :, 0].std(axis=1) / np.sqrt(scores_creative_chat.shape[1]), alpha=0.2, color="lightblue")

        ax.plot(temperatures, scores_creative_plm[:, :, 0].mean(axis=1), label='llama2 creative', marker='o', linestyle='--', color="salmon")
        ax.fill_between(temperatures, scores_creative_plm[:, :, 0].mean(axis=1) - 2 * scores_creative_plm[:, :, 0].std(axis=1) / np.sqrt(scores_creative_plm.shape[1]), 
                        scores_creative_plm[:, :, 0].mean(axis=1) + 2 * scores_creative_plm[:, :, 0].std(axis=1) / np.sqrt(scores_creative_plm.shape[1]), alpha=0.2, color="salmon")

        ax.set_xlabel('Temperature')
        # ax.set_ylabel(metric)
        ax.set_ylabel(metric_label[i])
        # ax.set_title(f'Diversity metric: {metric} for {20 if col_idx == 0 else 5} max words')
        #ax.legend()
        text_effect = [path_effects.withStroke(linewidth=3, foreground='white')]

        # Add labels directly on the plot with white outline
        end_temp = temperatures[-4]
        text_factual1 = ax.text(end_temp, scores_factual_chat[:, :, 0].mean(axis=1)[-4], 'llama2-chat factual', color='blue', verticalalignment='bottom')
        text_factual1.set_path_effects(text_effect)

        text_factual2 = ax.text(end_temp, scores_factual_plm[:, :, 0].mean(axis=1)[-4], 'llama2 factual', color='red', verticalalignment='top')
        text_factual2.set_path_effects(text_effect)

        text_creative1 = ax.text(end_temp, scores_creative_chat[:, :, 0].mean(axis=1)[-4], 'llama2-chat creative', color='lightblue', verticalalignment='top', style='italic')
        text_creative1.set_path_effects(text_effect)

        text_creative2 = ax.text(end_temp, scores_creative_plm[:, :, 0].mean(axis=1)[-4], 'llama2 creative', color='salmon', verticalalignment='bottom', style='italic')
        text_creative2.set_path_effects(text_effect)
        i+=1


plt.tight_layout()
plt.show()

fig.savefig(creative_factual_full_f, bbox_inches="tight") 
plt.close()

## Plot the difference in diversity between creative and factual changes for llama2 and llama2-chat

In [ ]:
num_metrics = len(diversity_metrics.keys())
fig, axes = plt.subplots(num_metrics, 2, figsize=(20, 5*num_metrics))
# make font bigger
plt.rcParams.update({'font.size': 16})
text_effect = [path_effects.withStroke(linewidth=3, foreground='white')]

i=0
for index, metric in enumerate(diversity_metrics.keys()):

    for col_idx, result_chat in enumerate([llama2_chat_results_20_words, llama2_chat_results_5_words]):
        ax = axes[index, col_idx]

        scores_factual_chat = result_chat["factual"][metric]
        scores_creative_chat = result_chat["creative"][metric]

        result_plm = [llama2_results_20_words, llama2_results_5_words][col_idx]
        scores_factual_plm = result_plm["factual"][metric]
        scores_creative_plm = result_plm["creative"][metric]

        # Calculate differences
        diff_llamaa_chat = scores_creative_chat[:, :, 0] - scores_factual_chat[:, :, 0]
        diff_llamaa_plm = scores_creative_plm[:, :, 0] - scores_factual_plm[:, :, 0]

        # Plotting differences
        ax.plot(temperatures, diff_llamaa_chat.mean(axis=1), label='llama2-chat', marker='o', color="blue")
        ax.fill_between(temperatures, diff_llamaa_chat.mean(axis=1) - 2 * diff_llamaa_chat.std(axis=1) / np.sqrt(diff_llamaa_chat.shape[1]), 
                        diff_llamaa_chat.mean(axis=1) + 2 * diff_llamaa_chat.std(axis=1) / np.sqrt(diff_llamaa_chat.shape[1]), alpha=0.2, color="blue")

        ax.plot(temperatures, diff_llamaa_plm.mean(axis=1), label='llama2', marker='o', color="red")
        ax.fill_between(temperatures, diff_llamaa_plm.mean(axis=1) - 2 * diff_llamaa_plm.std(axis=1) / np.sqrt(diff_llamaa_plm.shape[1]), 
                        diff_llamaa_plm.mean(axis=1) + 2 * diff_llamaa_plm.std(axis=1) / np.sqrt(diff_llamaa_plm.shape[1]), alpha=0.2, color="red")

        end_temp = temperatures[-4]
        text_llama = ax.text(end_temp, diff_llamaa_chat.mean(axis=1)[-4], 'llama2-chat', color='blue', verticalalignment='bottom')
        text_llama.set_path_effects(text_effect)

        text_vicuna = ax.text(end_temp, diff_llamaa_plm.mean(axis=1)[-4], 'llama2', color='red', verticalalignment='top')
        text_vicuna.set_path_effects(text_effect)

        # ax.set_title(f"Metric: {metric} for Result {20 if col_idx == 0 else 5} max words")
        ax.set_xlabel("Temperature")
        ax.set_ylabel(f"Creative {metric} - Factual {metric}")
        i+=1

plt.tight_layout()
plt.show()